In [1]:
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [6]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) # To be changed to 14

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2019, 6, 29), datetime.date(2019, 6, 22)]

In [7]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


In [8]:
new_2_biweekly_files

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormMasterWeekly20190625-111845-768.txt',
 '/home/jian/BigLots/MediaStorm_2019-06-29/MediaStormMasterWeekly20190702-113206-728.txt']

In [9]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))

236692
236683


In [10]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

72


In [11]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

In [12]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2019-07-08 13:59:56.127191 236680
2019-07-08 13:59:58.182970 236674
2019-07-08 13:59:58.826422 236674
2019-07-08 13:59:59.573165 236672
2019-07-08 14:00:00.181912 236672
2019-07-08 14:00:00.331816 236672
2019-07-08 14:00:00.456755 236672
2019-07-08 14:00:00.526470 236672
2019-07-08 14:00:00.583527 236672
2019-07-08 14:00:00.640574 236672
2019-07-08 14:00:00.738942 236672
2019-07-08 14:00:00.847862 236672
2019-07-08 14:00:00.908412 236672
2019-07-08 14:00:00.974043 236672
2019-07-08 14:00:01.035004 236672
2019-07-08 14:00:01.087854 236672
2019-07-08 14:00:01.200313 236671
2019-07-08 14:00:01.298283 236671
2019-07-08 14:00:01.349072 236671
2019-07-08 14:00:01.416695 236671
2019-07-08 14:00:01.467108 236671
2019-07-08 14:00:01.711358 236671
2019-07-08 14:00:01.923220 236671
2019-07-08 14:00:02.012310 236671
2019-07-08 14:00:02.098375 236671
2019-07-08 14:00:02.151717 236671
2019-07-08 14:00:06.369598 236669
2019-07-08 14:00:08.260749 236665
2019-07-08 14:00:14.419256 236663
2019-07-08 14:

In [13]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

236620


In [14]:
zipmap = pd.read_csv('/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/zip_with_ta_dma.csv',dtype = 'str')
zipmap['zipcodegroup'] = zipmap['revenue_flag']
zipmap = zipmap[['zip','zipcodegroup']].drop_duplicates('zip')
zipmap.columns = ['customer_zip_code','zipcodegroup']
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['S' 'P' nan 'T']
['S' 'P' 'T']
P 126281
S 32116
T 78223


In [15]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,126281
1,S,32116
2,T,78223


In [16]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment
4,b2f02a0b6ac6d835d570122eac60f9e9c562f58136045d...,42e661f2f7dc149ee14f6ac8aa272843e0c1bec608d796...,NaN,NewReward_062919_T
5,1357dd099a168b0f42b698a9a04aca395c9ed039b97d16...,56020b811573e6c045675016bbbc4f1e2c4da7693f3434...,NaN,NewReward_062919_T


In [17]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
